In [2]:
import numpy as np
import pandas as pd

import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import layers
from keras.layers import *
from keras.utils import np_utils

from keras_tqdm import TQDMNotebookCallback


In [1]:
pip install keras-tqdm

In [3]:
#ignore warnings in the output
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [4]:
from tensorflow.python.client import device_lib

# Check all available devices if GPU is available
print(device_lib.list_local_devices())
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12975470899068218708
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14674281152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13309717602757190096
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:

train = pd.read_csv('/content/gdrive/MyDrive/Internship/emnist/emnist-balanced-train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/Internship/emnist/emnist-balanced-test.csv')
mapping = pd.read_csv("/content/gdrive/MyDrive/Internship/emnist/emnist-balanced-mapping.txt", 
                      delimiter = ' ', 
                      index_col=0, 
                      header=None, 
                      squeeze=True)

In [7]:
  
train.shape, test.shape, mapping.shape

((112799, 785), (18799, 785), (47,))

In [8]:

X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]

del train
del test

In [9]:
classes = np.unique(y_train)
num_classes = len(classes)
num_classes

47

In [10]:
# Convert the images into 3 channels
X_train=np.dstack([X_train] * 3)
X_test=np.dstack([X_test]*3)
X_train.shape,X_test.shape

((112799, 784, 3), (18799, 784, 3))

In [11]:
# Reshape images as per the tensor format required by tensorflow
X_train = X_train.reshape(-1, 28,28,3)
X_test= X_test.reshape (-1,28,28,3)
X_train.shape,X_test.shape

((112799, 28, 28, 3), (18799, 28, 28, 3))

In [12]:
# Resize the images 48*48 as required by VGG16
from keras.preprocessing.image import img_to_array, array_to_img
X_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_train])
X_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_test])
#train_x = preprocess_input(x)
X_train.shape, X_test.shape

((112799, 48, 48, 3), (18799, 48, 48, 3))

In [13]:
# Normalise the data and change data type
X_train = X_train / 255.
X_test = X_test / 255.
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [14]:
from keras.utils import to_categorical

# Converting Labels to one hot encoded format
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [15]:
# partition to train and val
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train, 
                                                  test_size= 0.10, 
                                                  random_state=88)

In [16]:
# Finally check the data size whether it is as per tensorflow and VGG16 requirement
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((101519, 48, 48, 3), (11280, 48, 48, 3), (101519,), (11280,))

In [17]:
# Define the parameters for instanitaing VGG16 model. 
IMG_WIDTH = 48
IMG_HEIGHT = 48
IMG_DEPTH = 3
BATCH_SIZE = 16

In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, time
import matplotlib.pyplot as plt
#from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
#from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16;
from keras.applications.vgg16 import preprocess_input
import os

In [19]:
# Preprocessing the input 
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)


In [21]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


model = Sequential()
model.add(Conv2D(input_shape=(48,48,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=47, activation="softmax"))

In [24]:
# Compile model
model.compile(loss='sparse_categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [ ]:
batch_size = 128
epochs = 100

# Fit model on training data
history = model.fit(X_train, y_train
              , batch_size
              , epochs
              , verbose=1
              , validation_data=(X_test, y_test)
             )

Epoch 1/100
794/794 [==============================] - ETA: 0s - loss: 4.5328 - accuracy: 0.0211

In [20]:
#  Create base model of VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False, 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH)
                 )
conv_base.summary()



58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
______________________________________________________________

In [21]:
# Extracting features
train_features = conv_base.predict(np.array(X_train), batch_size=BATCH_SIZE, verbose=1)
test_features = conv_base.predict(np.array(X_test), batch_size=BATCH_SIZE, verbose=1)
val_features = conv_base.predict(np.array(X_val), batch_size=BATCH_SIZE, verbose=1)
#for layer in conv_base.layers:
#    layer.trainable = False

705/705 [==============================] - 6s 8ms/step


In [22]:
# 6.1 Saving the features so that they can be used for future
np.savez("train_features", train_features, y_train)
np.savez("test_features", test_features, y_test)
np.savez("val_features", val_features, y_val)

In [23]:
# Current shape of features
print(train_features.shape, "\n",  test_features.shape, "\n", val_features.shape)

(101519, 1, 1, 512) 
 (18799, 1, 1, 512) 
 (11280, 1, 1, 512)


In [24]:
# Flatten extracted features
train_features_flat = np.reshape(train_features, (101519, 1*1*512))
test_features_flat = np.reshape(test_features, (18799, 1*1*512))
val_features_flat = np.reshape(val_features, (11280, 1*1*512))

In [25]:
from keras import models
from keras.models import Model
from keras import layers
from keras import optimizers
from keras import callbacks
from keras.layers.advanced_activations import LeakyReLU

In [27]:
# 7.0 Define the densely connected classifier followed by leakyrelu layer and finally dense layer for the number of classes
NB_TRAIN_SAMPLES = train_features_flat.shape[0]
NB_VALIDATION_SAMPLES = val_features_flat.shape[0]
NB_EPOCHS = 200

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=(1*1*512)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(num_classes, activation='softmax'))

In [28]:
# Compile the model.
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(),
  # optimizer=optimizers.RMSprop(lr=2e-5),
    metrics=['acc'])

In [29]:
# Incorporating reduced learning and early stopping for callback
reduce_learning = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    verbose=1,
    mode='auto',
    epsilon=0.0001,
    cooldown=2,
    min_lr=0)
"""
eary_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=1,
    mode='auto')

callbacks = [reduce_learning, eary_stopping]"""

"\neary_stopping = callbacks.EarlyStopping(\n    monitor='val_loss',\n    min_delta=0,\n    patience=7,\n    verbose=1,\n    mode='auto')\n\ncallbacks = [reduce_learning, eary_stopping]"

In [30]:
y_train = to_categorical(y_train, 47)
y_test = to_categorical(y_test, 47)
y_val = to_categorical(y_val, 47)

In [31]:
# Train the the model
history = model.fit(
    train_features_flat,
    y_train,
    epochs=NB_EPOCHS,
    validation_data=(val_features_flat, y_val)
)

Epoch 1/200
3173/3173 [==============================] - 10s 3ms/step - loss: 3.1638 - acc: 0.1831 - val_loss: 2.0856 - val_acc: 0.3932
Epoch 2/200
3173/3173 [==============================] - 8s 2ms/step - loss: 1.9970 - acc: 0.4247 - val_loss: 1.7669 - val_acc: 0.4836
Epoch 3/200
3173/3173 [==============================] - 8s 2ms/step - loss: 1.7413 - acc: 0.4915 - val_loss: 1.6428 - val_acc: 0.5136
Epoch 4/200
3173/3173 [==============================] - 8s 2ms/step - loss: 1.6188 - acc: 0.5231 - val_loss: 1.5565 - val_acc: 0.5346
Epoch 5/200
3173/3173 [==============================] - 8s 2ms/step - loss: 1.5458 - acc: 0.5385 - val_loss: 1.5031 - val_acc: 0.5475
Epoch 6/200
3173/3173 [==============================] - 8s 2ms/step - loss: 1.5004 - acc: 0.5509 - val_loss: 1.4512 - val_acc: 0.5661
Epoch 7/200
3173/3173 [==============================] - 8s 2ms/step - loss: 1.4682 - acc: 0.5628 - val_loss: 1.4342 - val_acc: 0.5690
Epoch 8/200
3173/3173 [==============================]